In [18]:
import pandas as pd
import numpy as np
import difflib

In [19]:
year = 2013
df = pd.read_csv(f'./Raw/{year}.csv', sep=';', encoding ='latin')
df_processed = df[['DS_RESUMO', 'DS_PALAVRA_CHAVE', 'NM_AREA_AVALIACAO']].head(100).copy()

In [20]:
df_processed.columns = df_processed.columns.str.slice(3).str.lower()
df_processed.rename(columns={
    'base': 'ano'
}, inplace=True)

for item in list(df_processed.iloc[:, 0:].columns):
    df_processed[f'{item}'] = df_processed[f'{item}'].str.lower()

In [21]:
def rename_areas(text: str) -> str:
    if text in ['administração pública e de empresas, ciências contábeis e turismo', 'administração, ciências contábeis e turismo']:
        return 'administração pública e de empresas, ciências contábeis e turismo'
    elif text in ['arquitetura e urbanismo', 'arquitetura, urbanismo e design']:
        return 'arquitetura, urbanismo e design'
    elif text in 'filosofia/teologia:subcomissão filosofia':
        return 'filosofia'
    elif text in ['filosofia/teologia:subcomissão teologia', 'teologia']:
        return 'ciências da religião e teologia'
    elif text in 'letras / linguística':
        return 'linguística e literatura'
    elif text in 'artes / música':
        return 'artes'
    elif text in 'ciências sociais aplicadas i':
        return 'serviço social'
    else:
        return text

In [22]:
df_processed.area_avaliacao = df_processed.area_avaliacao.apply(rename_areas)
del rename_areas

In [23]:
colegios_dict = {
    'ciências da vida': {
        'ciências agrárias': ['ciência de alimentos', 'ciências agrárias i', 'medicina veterinária', 'zootecnia / recursos pesqueiros'],
        'ciências biológicas': ['biodiversidade', 'ciências biológicas i', 'ciências biológicas ii', 'ciências biológicas iii'],
        'ciências da saúde': ['educação física', 'enfermagem', 'farmácia', 'medicina i', 'medicina ii', 'medicina iii', 'nutrição', 'odontologia', 'saúde coletiva']
    },
    'humanidades': {
        'ciências humanas': ['antropologia / arqueologia', 'ciência política e relações internacionais', 'ciências da religião e teologia', 'educação', 'filosofia', 'geografia', 'história', 'psicologia', 'sociologia'],
        'ciências sociais aplicadas': ['administração pública e de empresas, ciências contábeis e turismo', 'arquitetura, urbanismo e design', 'comunicação e informação', 'direito', 'economia', 'planejamento urbano e regional / demografia', 'serviço social'],
        'linguística, letras e artes': ['artes', 'linguística e literatura']
    },
    'ciências exatas, tecnológicas e multidisciplinar': {
        'ciências exatas e da terra': ['astronomia / física', 'ciência da computação', 'geociências', 'matemática / probabilidade e estatística', 'química'],
        'engenharias': ['engenharias i', 'engenharias ii', 'engenharias iii', 'engenharias iv'],
        'multidisciplinar': ['biotecnologia', 'ciências ambientais', 'ensino', 'interdisciplinar', 'materiais']
    }
}

In [24]:
df_processed['colegios'] = ''
df_processed['areas'] = ''
df_processed.rename(columns={'area_avaliacao': 'subareas'}, inplace=True)

for colegios, areas in colegios_dict.items():
    for areas, subareas in areas.items():
        df_processed.loc[df_processed.subareas.isin(subareas), 'colegios'] = colegios
        df_processed.loc[df_processed.subareas.isin(subareas), 'areas'] = areas
        
df_processed = df_processed[['resumo', 'palavra_chave', 'colegios', 'areas', 'subareas']]

In [25]:
df_processed.palavra_chave.replace(to_replace = r'1[.]', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' \d[.]', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'(\S{2,})(?:\.)', value = '\\1;', regex=True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'\b(pt|en)\b', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' ,|, | [,] | /|/ | [/] ', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' [.] |\(\d\)', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' \x96 |\x93', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'[^a-z0-9\s\x80-\xff,.:;?!-]', value = '', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r' - |; | ;|; ;', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r';{2,}', value = ';', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace= r'[\W_]+$|^[;]+', value='', regex=True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'palavras-chave\S', value = '', regex = True, inplace=True)
df_processed.palavra_chave.replace(to_replace = r'^\s+|\s+$|\x80', value = '', regex = True, inplace=True)

In [26]:
df_processed.resumo.replace(to_replace= r'[^a-zA-Z0-9\s]+', value='', regex=True, inplace=True)

In [27]:
key_words = df_processed.palavra_chave.copy()
key_words = list(set(';'.join(key_words).split(';')))
key_words = [item for item in key_words if len(item) > 1]

for i in range (len(key_words)):
    key_words[i] = key_words[i].strip()
    
key_words = [item for item in key_words if len(item) < 80]

In [28]:
print(len(key_words))
print(len(set(key_words)))

385
385


In [29]:
def calculate_similarity_or_pertency(sentence: str, text: str) -> str:
    if sentence in text:
        return True
    similaridade = difflib.SequenceMatcher(None, sentence, text).ratio()
    porcentagem = int(similaridade * 100)
    if porcentagem > 80:
        return True
    
    return False

In [42]:
import dask.bag as db
import dask.distributed

def process_character(character):
    corte = df_processed[
        df_processed.palavra_chave.apply(
            lambda x: calculate_similarity_or_pertency(character, x)
        ) 
        |
        df_processed.resumo.apply(
            lambda x: calculate_similarity_or_pertency(character, x)
        )
    ].iloc[:, -3:]

    results = []
    for subarea in sorted(corte.subareas.unique()):
        colegio, area = (
            corte[corte.subareas == subarea].colegios.unique()[0],
            corte[corte.subareas == subarea].areas.unique()[0],
        )
        frequencia = corte.subareas.value_counts()[subarea]
        results.append([colegio, area, subarea, character, frequencia])

    return results

# criar um cliente Dask
client = dask.distributed.Client(address="10.0.0.115:8888", n_workers=2, threads_per_worker=16, memory_limit='10GB')

# criar uma bag a partir da lista key_words
bag = db.from_sequence(key_words)

# aplicar a função process_character em paralelo para cada elemento da bag
results_bag = bag.map(process_character)

# concatenar todos os resultados em uma única lista
results_list = results_bag.compute()

# fechar o cliente Dask
client.close()


/home/matheusmbr/anaconda3/envs/cluster_capes_venv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8888 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37027 instead
  warnings.warn(


In [40]:
freq = pd.DataFrame(results_list, columns=["colegio", "area", "subarea", "palavra_chave", "frequencia"])

2023-04-19 19:44:51,989 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)


ValueError: 5 columns passed, passed data had 9 columns

In [ ]:
freq.set_index('palavra_chave').sort_index().sort_values('frequencia', ascending=False)

,colegio,area,subarea,frequencia
palavra_chave,,,,
qu,humanidades,"linguística, letras e artes",linguística e literatura,38
literatura,humanidades,"linguística, letras e artes",linguística e literatura,21
qu,"ciências exatas, tecnológicas e multidisciplinar",multidisciplinar,interdisciplinar,19
contexto,humanidades,"linguística, letras e artes",linguística e literatura,13
narrativa,humanidades,"linguística, letras e artes",linguística e literatura,13
...,...,...,...,...
gerenciamento de redes,"ciências exatas, tecnológicas e multidisciplinar",ciências exatas e da terra,ciência da computação,1
fígado,ciências da vida,ciências da saúde,medicina i,1
fragmentação,humanidades,"linguística, letras e artes",linguística e literatura,1
